In [8]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import ipywidgets as widgets
from IPython.display import display, clear_output

# Define the plant: G(s) = 1 / (s + 1)
def plant():
    num = [1]
    den = [1, 1]
    return signal.TransferFunction(num, den)

# Define controller transfer functions
def controller_tf(controller_type, Kp=1.0, Ki=1.0, Kd=1.0):
    if controller_type == "P":
        num = [Kp]
        den = [1]
    elif controller_type == "I":
        num = [Ki]
        den = [1, 0]
    elif controller_type == "D":
        num = [Kd, 0]
        den = [1]
    elif controller_type == "PI":
        num = [Kp, Ki]
        den = [1, 0]
    elif controller_type == "PD":
        # Realistic derivative with low-pass filter: (Kd*s + Kp)/(τ*s + 1)
        tau = 0.01  # Small time constant to filter high-frequency noise
        num = [Kd, Kp]
        den = [tau, 1]

    elif controller_type == "PID":
        num = [Kd, Kp, Ki]
        den = [1, 0]
    else:
        raise ValueError("Invalid controller type")
    return signal.TransferFunction(num, den)

# Combine plant and controller in series and close the loop
def closed_loop_system(controller_type, Kp, Ki, Kd):
    G = plant()
    C = controller_tf(controller_type, Kp, Ki, Kd)
    open_loop = signal.TransferFunction(np.polymul(C.num, G.num),
                                        np.polymul(C.den, G.den))
    closed_loop = signal.TransferFunction(open_loop.num,
                                          np.polyadd(open_loop.den, open_loop.num))
    return open_loop, closed_loop

# Plotting functions
def plot_all(controller_type, Kp=1.0, Ki=1.0, Kd=1.0):
    open_loop, closed_loop = closed_loop_system(controller_type, Kp, Ki, Kd)
    
    # Time domain (step response)
    t, y = signal.step(closed_loop)
    
    # Frequency domain (Bode)
    w, mag, phase = signal.bode(open_loop)

    # Nyquist plot
    w_nyq = np.logspace(-2, 2, 1000)
    _, H = signal.freqresp(open_loop, w=w_nyq)

    # Plot
    plt.figure(figsize=(15, 10))

# Bode magnitude
    plt.subplot(2, 2, 2)
    plt.semilogx(w, mag)
    plt.title('Bode Magnitude')
    plt.xlabel('Frequency [rad/s]')
    plt.ylabel('Magnitude [dB]')
    plt.grid(True)

    # Step response
    plt.subplot(2, 2, 1)
    plt.plot(t, y)
    plt.title(f'Step Response - {controller_type}')
    plt.xlabel('Time [s]')
    plt.ylabel('Amplitude')
    plt.grid(True)


    # Bode phase
    plt.subplot(2, 2, 3)
    plt.semilogx(w, phase)
    plt.title('Bode Phase')
    plt.xlabel('Frequency [rad/s]')
    plt.ylabel('Phase [deg]')
    plt.grid(True)

        # Nyquist plot (corrected)
    plt.subplot(2, 2, 4)
    w_nyq = np.logspace(-2, 2, 1000)
    jw = 1j * w_nyq
    H = signal.freqresp(open_loop, w=w_nyq)[1]
    plt.plot(H.real, H.imag, label='Nyquist')
    plt.plot(H.real, -H.imag, '--', label='Mirror (Negative Freq)')
    plt.plot([-1], [0], 'ro', label='-1 (Critical Point)')
    plt.axhline(0, color='black', linewidth=0.5)
    plt.axvline(0, color='black', linewidth=0.5)
    plt.title('Nyquist Plot')
    plt.xlabel('Re')
    plt.ylabel('Im')
    plt.legend()
    plt.grid(True)


    plt.tight_layout()
    plt.show()

# Interactive widget
@widgets.interact(
    controller_type=widgets.Dropdown(options=['P', 'I', 'D', 'PI', 'PD', 'PID'], value='PID'),
    Kp=widgets.FloatSlider(min=0, max=10, step=0.1, value=1.0),
    Ki=widgets.FloatSlider(min=0, max=10, step=0.1, value=1.0),
    Kd=widgets.FloatSlider(min=0, max=10, step=0.1, value=1.0),
)
def interactive_plot(controller_type, Kp, Ki, Kd):
    clear_output(wait=True)
    plot_all(controller_type, Kp, Ki, Kd)


interactive(children=(Dropdown(description='controller_type', index=5, options=('P', 'I', 'D', 'PI', 'PD', 'PI…